In [2]:
import psi4
import numpy as np
import matplotlib.pyplot as plt
import fortecubeview as fview
import sys

## Is C3H+ Present in the Horsehead Nebula?
The purpose of this lab is to introduce you to how rotational spectroscopy can be used to detect molecules in the interstellar medium. Before beginning Part 1 of the experiment, please read over the Introduction in the Lab Manual (attached on Moodle). The remaining parts of the experiment are discussed both in the lab manual and in this Jupyter Notebook.

### Part 1: C3H+ B-type Rotational Constants

The code below creates a C3H+ molecule in Psi4. Note the syntax, in which the first line specifies the charge (+1) and multiplicity (1) for our molecule, and the remaining lines give the Cartesian coordintes in units of Angstroms. Also note that this first geometry is just a *guess*. We know that the molecule has to be linear, so we can create the molecule by giving its z-coordinate. C-C bond lengths of 1.5 Angstrom and C-H bond lengths of 1.0 Angstrom were used as rough initial guesses.

In [43]:
c3hplus = psi4.geometry("""
+1 1
C 0.0 0.0 0.0
C 0.0 0.0 1.5
C 0.0 0.0 3.0
H 0.0 0.0 4.0
""")

We can use a new (to us) Python package called fortecuveview (abbreviated fview) to visualize the results and make sure the molecule looks linear as expected. You can click and drag the molecule to rotate, or scroll to zoom in and out.

In [45]:
fview.geom(molecule=c3hplus)

Renderer(camera=OrthographicCamera(bottom=-5.0, children=(DirectionalLight(color='white', intensity=0.5, posit…

Next, we need to optimize the geometry of the molecule using the HF/STO-3G method. This can be directly called in Psi4 using the optimize function. Note that the optimize function changes the ch3plus molecule, so after this function call the ch3plus variable reflects the new geometry. 

For convenience and ease of visualization, the code below also asks Psi4 to print the output coordinates, and asks fview to generate a 3D model of the molecule.

In [46]:
psi4.optimize('hf/sto-3g',molecule=c3hplus)
psi4.set_output_file('c3h+_hf_sto3g.dat')
ch3plus.print_out_in_angstrom()
ch3plus.print_distances()
!cat 'c3h+_hf_sto3g.dat'
fview.geom(molecule=c3hplus)

Optimizer: Optimization complete!
    Molecular point group: c2v
    Full point group: C_inf_v

    Geometry (in Angstrom), charge = 1, multiplicity = 1:

       Center              X                  Y                   Z       
    ------------   -----------------  -----------------  -----------------
         C            0.000000000000     0.000000000000    -1.387018505166
         C           -0.000000000000     0.000000000000    -0.027891663493
         C           -0.000000000000     0.000000000000     1.220525413329
         H            0.000000000000     0.000000000000     2.314505980080

        Interatomic Distances (Angstroms)

        Distance 1 to 2 1.359   
        Distance 1 to 3 2.608   
        Distance 1 to 4 3.702   
        Distance 2 to 3 1.248   
        Distance 2 to 4 2.342   
        Distance 3 to 4 1.094   




Renderer(camera=OrthographicCamera(bottom=-5.0, children=(DirectionalLight(color='white', intensity=0.5, posit…

Our next job is to determine the rotational constants for this molecule. Psi4 makes this very easy! The Psi4 output is in wavenumbers, so you'll need to convert to MHz and GHz to complete the report sheet.

In [47]:
# Find rotational constants
rot_conts_wn = psi4.core.Molecule.rotational_constants(c3hplus).to_array() # in cm^-1
print(rot_conts_wn," in cm^-1")

# convert to MHz
c =  3e8 # speed of light in m/s
rot_conts_MHz = rot_conts_wn * 1e-6 * 1e2 * c
print(rot_conts_MHz," in MHz")

rot_conts_GHz = rot_conts_MHz * 1e-3
print(rot_conts_GHz," in GHz")

B_c3hplus_hf_sto3g = rot_conts_GHz[1]
print("For C3H+, B is ",B_c3hplus_hf_sto3g, "GHz")

[0.         0.36729697 0.36729697]  in cm^-1
[    0.         11018.90921156 11018.90921156]  in MHz
[ 0.         11.01890921 11.01890921]  in GHz
For C3H+, B is  11.018909211564216 GHz


*Activity:* Repeat the optimization for the other theory methods and basis sets discussed in the lab manual. The calculations with bigger basis sets and higher levels of theory can take a few minutes to run, so be patient.

In [48]:
psi4.optimize('hf/6-31G(d)',molecule=c3hplus)

Optimizer: Optimization complete!


-113.65462397617604

In [49]:
psi4.optimize('hf/cc-pVTZ',molecule=c3hplus)

Optimizer: Optimization complete!


-113.6884166889823

In [50]:
psi4.optimize('hf/cc-pVDZ',molecule=c3hplus)

Optimizer: Optimization complete!


-113.66041290266729

In [51]:
psi4.optimize('B3LYP/cc-pVDZ',molecule=c3hplus)

Optimizer: Optimization complete!


-114.34948604990613

In [52]:
psi4.optimize('MP2/cc-pVDZ',molecule=c3hplus)

Optimizer: Optimization complete!


-114.00051171489281

In [53]:
psi4.optimize('CCSD/cc-pVDZ',molecule=c3hplus)

Optimizer: Optimization complete!


-114.01970443431617

### Part 2: B-type rotational constants of other linear molecules
Is C3H+ the carrier of the rotational lines? Is another molecule a better carrier? Now, you will examine the B-type rotational constants of some other closed-shell linear molecules.

*Activity*: Create a new linear molecule: HCN. Your initial geometry guess doesn't have to be perfect, just reasonable. Make sure to input the appropriate charge and multiplicity for this molecule.

*Activity*: Perform a geometry optimization with B3LYP/cc-pVDZ. Using this geometry, determine the B rotational constant for HCN in units of GHz.

*Activity*: Repeat the analysis for each of the following linear molecules: HCCF, HC3N, C4H-, C2O. Add additional code blocks as desired to complete the activity.

### Part 3: Rotational Constants for non-linear molecules
Now you will compute the rotational constants of a few non-linear molecules and fit their rotational constants to the linear model in Eq 2. from the lab manual. 

Non-linear molecules have three rotational constants -- A, B, and C -- corresponding to the three rotational degrees of freedom for a non-linear molecule. By convention, A refers to the largest frequency motion (which is also the smallest energy motion in wavenumbers), and C to the smallest frequency motion. An approximate equation for the rotational energy levels of non-linear molecules is 

$$
E_J = (B+C)(J+1)
$$

The code below creates the geometry of HCCO-. This, and all molecules in Part 3, are "quasi-linear", meaning that the atoms are arranged in a chain, however the bond angles are not each 180° like would be expected for a linear molecule.

For technical reasons, if we give an initial geometry guess that is linear, Psi4 will try and optimize to a linear structure, even if that isn't the global minimum geometry. To avoid this issue, best practice is to slightly displace one of the coordinates (below H was chosen) in the x- or y-direction. Any roughly-reasonable guess should be fine.

In [57]:
hccominus = psi4.geometry("""
-1 1
H 0.0 0.5 0.0
C 0.0 0.0 1.0
C 0.0 0.0 2.5
O 0.0 0.0 4.0
""")

In [58]:
psi4.optimize('b3lyp/cc-pvdz',molecule=hccominus)
psi4.set_output_file('hccominus.dat')
hccominus.print_out_in_angstrom()
hccominus.print_distances()
!cat 'hccominus.dat'
fview.geom(molecule=hccominus)

Optimizer: Optimization complete!
    Molecular point group: cs
    Full point group: Cs

    Geometry (in Angstrom), charge = -1, multiplicity = 1:

       Center              X                  Y                   Z       
    ------------   -----------------  -----------------  -----------------
         H           -0.534291999546     2.170554900864     0.000000000000
         C            0.138231616116     1.318720718792     0.000000000000
         C           -0.041126647945     0.057818484086     0.000000000000
         O           -0.039186627831    -1.169497333519     0.000000000000

        Interatomic Distances (Angstroms)

        Distance 1 to 2 1.085   
        Distance 1 to 3 2.170   
        Distance 1 to 4 3.377   
        Distance 2 to 3 1.274   
        Distance 2 to 4 2.495   
        Distance 3 to 4 1.227   


                                                                                                                                                            

Renderer(camera=OrthographicCamera(bottom=-5.0, children=(DirectionalLight(color='white', intensity=0.5, posit…

*Activity*: Find the effective rotational constant, $B_{eff}$, for HCCO-. 

In [59]:
# Find rotational constants
rot_conts_wn = psi4.core.Molecule.rotational_constants(hccominus).to_array() # in cm^-1
print(rot_conts_wn," in cm^-1")

# convert to MHz
c =  3e8 # speed of light in m/s
rot_conts_MHz = rot_conts_wn * 1e-6 * 1e2 * c
print(rot_conts_MHz," in MHz")

rot_conts_GHz = rot_conts_MHz * 1e-3
print(rot_conts_GHz," in GHz")

Beff_hccominus = 0.5*(rot_conts_GHz[1]+rot_conts_GHz[2])
print("For HCCO-, Beff is ",Beff_hccominus, "GHz")

[33.80144019  0.35417909  0.35050641]  in cm^-1
[1014043.20583716   10625.37274041   10515.19219206]  in MHz
[1014.04320584   10.62537274   10.51519219]  in GHz
For HCCO-, Beff is  10.57028246623479 GHz


*Activity*: Determine the rotational constants (A, B, C, and Beff) for the following non-linear molecules: NNOH+, HOCO+, C3H-, HCCN, and HOCN. Add additional code blocks to your notebook as needed. 

### Report and Summary

#### Part 1: C3H+ B-type Rotational Constants

Fill out the table of rotational constants (in GHz) for C3H+ using different methods and basis sets. (Note: you can edit this table directly by double clicking on this text).

| Method/Basis Set     | B (GHz) |
| ----------- | ----------- |
| HF/STO-3G        |        |
| HF/6-31G(d)      |        |
| HF/cc-pVTZ       |        |
| HF/cc-pVDZ       |        |
| B3LLYP/cc-pVDZ   |        |
| MP2/cc-pVDZ      |        |
| CCSD/cc-pVDZ     |        |


*Question*: How does the change in basis set affect the primary rotational constant for C3H+?

*Question*: How does the change in method affect the primary rotational constant for this molecule?

*Question*: Compared to the experimental value, what is the most accurate method? The most accurate basis set? What would be the most accurate method/basis set combination?

*Question*: Would you expect this combination to give the most accurate result? Why or why not?

#### Part 2: B-type rotational constants for linear molecules

Fill out the table of rotational constants (in GHz) for the molecules below based on your B3LYP/cc-pVDZ calculations.

| Molecule     | B (GHz) |
| ----------- | ----------- |
| HCCN        |        |
| HCCF        |        |
| HC3N        |        |
| C4H-        |        |
| C2O         |        |



#### Part 3: Rotational constants for non-linear molecules

Fill out the table of rotational constants (in GHz) for the non-linear molecules below.

| Molecule     | A (GHz) |B (GHz) |C (GHz) |Beff (GHz) |
| -----------  | ------- |------- |------- |------- |
| HCCO-        |         |        |        |        |
| NNOH+        |         |        |        |        |
| HOCO+        |         |        |        |        |
| C3H-         |         |        |        |        |
| HCCN         |         |        |        |        |
| HOCN         |         |        |        |        |


*Question*: You may have noticed that all of the molecules chosen, whether they are cations, anions, or neutrals, are all closed-shell.  The reason comes from the very close fine-splitting observed in rotational lines of open-shell molecules. The lines observed did not have this splitting present. This is one way to eliminate carriers of lines.  Another is simply the calculated rotational constants themselves. How does this let you eliminate some of the above molecules as carriers, and what molecules did you easily eliminate?

*Question*: Compare all of the B3LYP/cc-pVDZ $B$ (for linear molecules) and $B_{eff}$ values (for non-linear molecules). Which three have the closest B-type constant to that from the lines originally observed in Horsehead nebula PDR?

*Question*: What molecule do you believe is the carrier of these astronomical rotational lines? Why, and what may you not be considering (Hint: Don’t forget $A$)?

*Summarize*: In 500 words or less, summarize the objective, procedure, main results, and conclusion(s) from today's experiment.